In [ ]:
import os

import evaluate
import numpy as np
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    logging,
)

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

model_name = "distilbert-base-uncased"
max_length = 512
dataset_name = "stanfordnlp/sst2"
metric = evaluate.load("accuracy")

In [3]:
dataset = load_dataset(dataset_name)

train_texts = dataset["train"]["sentence"]
train_labels = dataset["train"]["label"]

val_texts = dataset["validation"]["sentence"]
val_labels = dataset["validation"]["label"]

KeyboardInterrupt: 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(
    train_texts, truncation=True, padding=True, max_length=max_length
)
val_encodings = tokenizer(
    val_texts, truncation=True, padding=True, max_length=max_length
)

In [ ]:
class SSTDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = SSTDataset(train_encodings, train_labels)
eval_dataset = SSTDataset(val_encodings, val_labels)

In [ ]:
# Ignore warning on model loading
logging.set_verbosity_error()
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(
    output_dir="./test_trainer",
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs",
    dataloader_pin_memory=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
accuracy_score = trainer.evaluate()["eval_accuracy"]
print(f"\nAccuracy : {round(accuracy_score, 3)}")

{'eval_loss': 0.6933807730674744, 'eval_model_preparation_time': 0.0008, 'eval_accuracy': 0.49655963302752293, 'eval_runtime': 1.6821, 'eval_samples_per_second': 518.402, 'eval_steps_per_second': 8.323}
Accuracy : 0.497


In [ ]:
trainer.train()

KeyboardInterrupt: 

In [ ]:
accuracy_score = trainer.evaluate()["eval_accuracy"]
print(f"\nAccuracy : {round(accuracy_score, 3)}")

/Users/vladimir/Desktop/language-models-exploration/venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.40041056275367737, 'eval_accuracy': 0.8176605504587156, 'eval_runtime': 1.6371, 'eval_samples_per_second': 532.639, 'eval_steps_per_second': 8.552, 'epoch': 0.024228028503562947}
0.818
